# q3

In [ ]:
!mkdir data1
!echo "dog cat cat apple ninja" > data1/text1.txt
!echo "ninja apple cat keyboard horse" > data1/another_file.txt
!echo "water cat dog spider spider" > data1/yet_another_file.txt

In [ ]:
!hadoop fs -mkdir -p /outputs
!hadoop fs -mkdir -p /inputs
!hadoop fs -put data1 /inputs

In [ ]:
!hadoop dfs -ls /inputs/data1

In [ ]:
%%writefile mapper.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*
import sys
import string
import os
import ntpath

for line in sys.stdin: # reads from stdin
    line = line.strip()
    words = line.split()

    for word in words: # writes to stdout
        word = word.translate(str.maketrans('','',string.punctuation))
        print(word.lower()+'\t'+ntpath.basename(os.getenv('mapreduce_map_input_file')))

In [ ]:
%%writefile reducer.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*

import sys
import random

counts = dict()
for line in sys.stdin:
    line = line.strip()
    word, file = line.split()
    if word in counts:
        counts[word] += "\t" + file
    else:
        counts[word] = file
        
for key in counts:
    print(key +"\t"+ counts[key])

In [ ]:
!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -file $PWD/mapper.py\
    -file $PWD/reducer.py\
    -mapper mapper.py \
    -reducer reducer.py \
    -input /inputs/data1 \
    -output /outputs/result3

In [ ]:
!hdfs dfs -rm -R /outputs/result3

In [ ]:
!hdfs dfs -ls /outputs/result3

I originally had big file for all the questions. But I realized they needed to be split. So This result is from a old run.

In [2]:
!hdfs dfs -cat /outputs/result3/part-00000

2020-10-16 23:11:32,659 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
apple	text1.txt	another_file.txt
cat	another_file.txt	text1.txt	text1.txt	yet_another_file.txt
dog	yet_another_file.txt	text1.txt
horse	another_file.txt
keyboard	another_file.txt
ninja	another_file.txt	text1.txt
spider	yet_another_file.txt	yet_another_file.txt
water	yet_another_file.txt
